In [ ]:
# Global import

import datetime as dt

import pandas as pd
import numpy as np
import plotly.graph_objects as go

from dash_mdc_neptune import themes

In [ ]:
# Global variables

ASSET_NAME = "PWRTE"
PORTFOLIO_NAME = "TEST_FR"

COLUMN_NAMES = npta.definitions.TRADE_DATE_COLUMNS

# PAGE 1

In [ ]:
# Retrieve trades

query_params = {
    "asset": ASSET_NAME,
    "portfolio_name": PORTFOLIO_NAME,
}

trades_df = nptc.filter_forward_contracts(**query_params)

## SECTION 1

### CARD 1

In [ ]:
# Format trades

all_df = trades_df[COLUMN_NAMES]
latest_df = all_df[all_df.traded_at==all_df.traded_at.max()]

### CARD 2

In [ ]:
all_volume_df = npta.get_volumes_and_costs_from_trades(all_df)
latest_volume_df = npta.get_volumes_and_costs_from_trades(latest_df)

In [ ]:
# Shared heatmap configs

GO_LAYOUT = go.Layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='local delivery date',
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='MWh',
        ),
        side="left",
        overlaying='y2',
    ),
    yaxis2=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='EUR/MWh',
        ),
        side="right",
    ),
    template="noos_watermark+noos_base+hoover_xunified+min_modebar",
)

In [ ]:
all_volume = go.Scatter(
    x = all_volume_df.index,
    y = all_volume_df.total_volume * 2,
    line_color=themes.NOOS_HTML_COLORS["gray"],
    name = "Volumes",
    fill='tozeroy',
    yaxis="y",
    visible = True
)

all_price = go.Scatter(
    x = all_volume_df.index,
    y = all_volume_df.price_average,
    line_color=themes.NOOS_HTML_COLORS["accent"],
    name = "Prices",
    visible = True,
    yaxis="y2"
)

latest_volume = go.Scatter(
    x = latest_volume_df.index,
    y = latest_volume_df.total_volume * 2,
    line_color=themes.NOOS_HTML_COLORS["gray"],
    name = "Volumes",
    fill='tozeroy',
    yaxis="y",
    visible = False
)

latest_price = go.Scatter(
    x = latest_volume_df.index,
    y = latest_volume_df.price_average,
    line_color=themes.NOOS_HTML_COLORS["accent"],
    name = "Prices",
    visible = False,
    yaxis="y2"
)

fig = go.FigureWidget(
    data=[
        all_volume,
        all_price,
        latest_volume,
        latest_price,
    ],
    layout=GO_LAYOUT,
)

# DASH

In [ ]:
import jupyter_dash

from dash import dcc

import dash_mdc_neptune as mdc

## LAYOUT

In [ ]:
# Display graph & table in tabs
graph = dcc.Graph(
    id="graph",
    figure=fig,
    style={"height": "100%", "width": "100%"},
)
table = mdc.Table(
    id="table",
    columns=[{"field": c.replace("_", " ").capitalize(), "width": 200} for c in all_df.columns],
    rows=all_df.to_dict(orient='records'),
)

tab = mdc.Tab(
    children=[table, graph],
    tabs=[
        {"label": "Trade blotter"},
        {"label": "Trade profile"},
    ],
)

In [ ]:
# Visualize based on toggles
toggle = mdc.Toggle(
    id="toggle",
    options=['All', 'Last'],
)

box = mdc.Box(children=[toggle, tab])

In [ ]:
# Dashboard layout
section = mdc.Section(
    children=box,
    cards=[{"title": "Custom portfolio"}],
)

page = mdc.Page(children=section)
navbar = mdc.NavBar(title="Wholesale Hedge Trades")

layout = mdc.Dashboard(children=[navbar, page])

## APP

In [ ]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output


app = JupyterDash(__name__)
app.layout = layout


@app.callback(
    Output(component_id='table', component_property='rows'),
    Output(component_id='graph', component_property='figure'),
    Input(component_id='toggle', component_property='selected'),
)
def update_tabs(value):
    if value == 0:
        rows = all_df.to_dict('records')
        fig.data[2].visible=False
        fig.data[3].visible=False
        fig.data[0].visible=True
        fig.data[1].visible=True
    else:
        rows = latest_df.to_dict('records')
        fig.data[0].visible=False
        fig.data[1].visible=False
        fig.data[2].visible=True
        fig.data[3].visible=True
    return [rows, fig]

app.run_server(mode='external', host="0.0.0.0", port=8001)